## 딥러닝 실습 과제 2주차 - CNN을 활용하여 보드 전체를 입력받아 9칸 상태 예측

다음  세 가지 활동을 해봅시다.

01. **모델 설계**: CNN 기반 분류 모델 활용
02. **손실 함수 정의**: 각 칸에 대해 3-클래스 분류
03. **학습 및 평가**: 시각화 함수, 정확도 측정, confusion matrix 등

TTTDataset.zip을 불러와 문제에서 요하는 코드를 구현하세요.

💡 **데이터 구조**  
- **`image_black`** : 이미지 데이터  
- **`labels`** : 타겟 데이터  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import json
import glob
import os

## 00. 클래스
정의한 클래스를 이용해 실행해 주세요.

In [3]:
class TTTDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        """
        틱택토 데이터셋을 PyTorch Dataset 형태로 변환.
        :param image_paths: 이미지 파일 경로 리스트
        :param label_paths: 레이블 JSON 파일 경로 리스트
        :param transform: 이미지 전처리 변환
        """
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform
        self.data = self._load_data()


    def _load_data(self):
        """ 이미지 & 레이블 로드 """
        data = []
        for img_path, lbl_path in zip(self.image_paths, self.label_paths):
            # 이미지를 흑백(Grayscale)로 변환
            image = Image.open(img_path).convert("L")  # "RGB" 대신 "L" 사용

            # JSON 레이블 로드
            with open(lbl_path, 'r') as f:
                labels = json.load(f)

            # 레이블을 숫자로 변환 (O=1, X=-1, blank=0)
            label_tensor = torch.tensor(
                [1 if v == "O" else -1 if v == "X" else 0 for v in labels.values()],
                dtype=torch.float32
            )
            data.append((image, label_tensor))

        return data


    def __len__(self):
        """ 데이터셋 크기 반환 """
        return len(self.data)


    def __getitem__(self, idx):
        """ 데이터셋에서 idx 번째 샘플(이미지 & 레이블)을 가져오는 역할 """
        image, label = self.data[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
# 압축 파일 풀기
zip_path = "/content/drive/MyDrive/TTTDataset.zip"
extract_path = "/content/TTTDataset"

if not os.path.exists(extract_path):
    !unzip -q "{zip_path}" -d "/content/"
    print(" 압축 해제 완료!")
else:
    print(" 이미 압축이 풀려 있으므로 건너뜁니다.")

 압축 해제 완료!


In [5]:
# 이미지 및 레이블 디렉토리 경로 설정
image_dir = "/content/TTTDataset/image_black"
label_dir = "/content/TTTDataset/labels"

# 이미지와 라벨 파일 자동 수집 (확장자가 .jpg 또는 .JPG인 점 확인)
image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")) +
                     glob.glob(os.path.join(image_dir, "*.JPG")))

label_paths = sorted(glob.glob(os.path.join(label_dir, "*.json")))

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(), # 픽셀값을 텐서로 변환 (0~1로 자동 스케일링됨)
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Dataset 생성
dataset = TTTDataset(image_paths, label_paths, transform=transform)

# 전체 데이터 크기 기준 분할 비율 설정 (예: 70% train, 15% val, 15% test)
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# 데이터 분할
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# 각각에 대한 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 01. 모델 설계: CNN 기반 분류 모델 활용
- 입력: 틱택토 보드 전체 이미지 (예: 128x128)
- 출력: 9개의 상태(각 칸마다 O, X, blank 중 하나)

In [6]:
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [7]:
class TicTacToeCNN(nn.Module):
    def __init__(self):
        # super(): CNN class의 부모 class인 nn.Module을 초기화
        super(TicTacToeCNN, self).__init__()

        # 첫번째 층
        self.layer1 = nn.Sequential(
            # Convolution Layer: 1채널 입력을 32채널 특징맵(feature map)으로 변환
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),  # (B, 1, 128, 128) → (B, 16, 128, 128)
            # 비선형 활성함수
            nn.ReLU(),
            # Pooling Layer: 크기 절반으로 축소
            nn.MaxPool2d(kernel_size=2, stride=2))            # (B, 32, 64, 64)

        # 두번째 층
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))              # (B, 64, 32, 32)

        # 세번째 층
        self.layer3= nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))    # (B, 128, 1, 1)
        )

        # Fully-connected Layer: 칸 별로 3-클래스 점수 출력
        self.fc_layer = nn.Sequential(
            nn.Linear(128, 256),   # 중간 hidden layer 추가
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 9 * 3))   # 128차원 백터 -> 27개 출력 (9칸 × 각 칸 당 3-클래스)


    def forward(self, x):
        x = self.layer1(x)              # (B, 16, 64, 64)
        x = self.layer2(x)              # (B, 32, 32, 32)
        x = self.layer3(x)              # (B, 64, 1, 1)
        x = x.view(x.size(0), -1)       # (B, 64)
        x = self.fc_layer(x)            # (B, 27)
        return x.view(-1, 9, 3)         # (B, 9, 3)

## 02. 손실 함수 정의: 각 칸에 대해 3-클래스 분류
- 다중 클래스 분류이므로 CrossEntropyLoss 사용 가능
- 9개 칸을 각각 분류하는 방식으로 모델 구성

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 모델 객체 선언
model = TicTacToeCNN().to(device)

# 손실 함수 정의 (다중 클래스 분류)
criterion = nn.CrossEntropyLoss()
# 최적화 알고리즘 정의
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## 03. 학습 및 평가: 시각화 함수, 정확도 측정, confusion matrix 등
- 학습 데이터로 모델을 훈련하고, 정확도 측정
- 예측이 잘 되는지 시각화하여 분석

In [9]:
# 전처리 함수 정의
def preprocess_batch(images, labels):
    images = images.to(device)
    labels = labels.to(device)
    # CrossEntropy는 정수 클래스 (0, 1, 2)를 기대함
    target = (labels + 1).to(torch.int64)   # -1 → 0, 0 → 1, 1 → 2로 정수형 변환
    return images, target

In [10]:
# 평가 함수 정의
def evaluate(loader):  # train_loader, val_loader, test_loader 중 하나를 넘길 수 있음
    model.eval()
    # 예측이 맞은 칸 수
    correct = 0
    # 전체 칸 수
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, target = preprocess_batch(images, labels)
            # 모델에 이미지 삽입하여 예측 수행
            outputs = model(images)  # (B, 9, 3)
            # 각 칸마다 3-클래스 중 가장 높은 점수를 선택
            preds = outputs.argmax(dim=2)  # (B, 9)
            correct += (preds == target).sum().item()
            total += target.numel()

    return correct / total

In [11]:
for epoch in range(100):
    model.train()
    # 누적 손실 초기화
    total_loss = 0

    for images, labels in train_loader:
        images, target = preprocess_batch(images, labels)

        optimizer.zero_grad() # 기울기 초기화
        outputs = model(images)  # (B, 9, 3)
        loss = criterion(outputs.view(-1, 3), target.view(-1)) # 손실 계산
        loss.backward() # 역전파
        optimizer.step() # 가중치 업데이트
        total_loss += loss.item() # 손실 누적

    # 1 epoch마다 검증 정확도 평가
    val_acc = evaluate(val_loader)
    print(f"[Epoch {epoch+1:02d}] Loss: {total_loss:.4f} | Val Accuracy: {val_acc:.2%}")

[Epoch 01] Loss: 10.9935 | Val Accuracy: 38.14%
[Epoch 02] Loss: 10.8863 | Val Accuracy: 41.13%
[Epoch 03] Loss: 10.8382 | Val Accuracy: 39.30%
[Epoch 04] Loss: 10.7715 | Val Accuracy: 43.12%
[Epoch 05] Loss: 10.6664 | Val Accuracy: 41.46%
[Epoch 06] Loss: 10.6633 | Val Accuracy: 40.63%
[Epoch 07] Loss: 10.5939 | Val Accuracy: 47.26%
[Epoch 08] Loss: 10.4820 | Val Accuracy: 47.76%
[Epoch 09] Loss: 10.4154 | Val Accuracy: 46.60%
[Epoch 10] Loss: 10.2621 | Val Accuracy: 45.77%
[Epoch 11] Loss: 10.1795 | Val Accuracy: 46.43%
[Epoch 12] Loss: 10.1407 | Val Accuracy: 45.77%
[Epoch 13] Loss: 10.2034 | Val Accuracy: 44.78%
[Epoch 14] Loss: 10.0525 | Val Accuracy: 44.78%
[Epoch 15] Loss: 9.9779 | Val Accuracy: 45.61%
[Epoch 16] Loss: 9.8830 | Val Accuracy: 45.94%
[Epoch 17] Loss: 9.8448 | Val Accuracy: 46.43%
[Epoch 18] Loss: 9.7662 | Val Accuracy: 46.77%
[Epoch 19] Loss: 9.6980 | Val Accuracy: 45.61%
[Epoch 20] Loss: 9.6574 | Val Accuracy: 44.94%
[Epoch 21] Loss: 9.5902 | Val Accuracy: 46.43%

In [12]:
test_acc = evaluate(test_loader)
print(f"Test Accuracy: {test_acc:.2%}")

Test Accuracy: 53.78%
